In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

In [4]:

# ================= 绘图风格设置 =================
fonts = ['SimHei', 'Microsoft YaHei', 'SimSun', 'Arial Unicode MS']
found_font = False
for font in fonts:
    try:
        mpl.rcParams['font.sans-serif'] = [font] + mpl.rcParams['font.sans-serif']
        found_font = True
        break
    except:
        continue

mpl.rcParams['axes.unicode_minus'] = False
plt.style.use('ggplot')


In [5]:

# ================= 路径设置 (兼容 Notebook) =================
try:
    # 脚本模式
    CUR_DIR = os.path.dirname(os.path.abspath(__file__))
except NameError:
    # Notebook 模式
    current_cwd = os.getcwd()
    # 尝试定位数据文件
    candidates = [
        ".", 
        "results_analysis/fairmot_metrics",
        "fairmot_metrics"
    ]
    target_dir = current_cwd
    for path in candidates:
        full_path = os.path.join(current_cwd, path)
        if os.path.exists(os.path.join(full_path, "fairmot_all_metrics.csv")):
            target_dir = full_path
            break
    CUR_DIR = target_dir

print(f"当前工作目录: {CUR_DIR}")

CSV_PATH = os.path.join(CUR_DIR, "fairmot_all_metrics.csv")
OUT_DIR = os.path.join(CUR_DIR, "figures")
os.makedirs(OUT_DIR, exist_ok=True)


当前工作目录: D:\Data\25-26一\图像工程\work\results_analysis\fairmot_metrics\.


In [6]:

# -------------------------------------------------
# 读取数据
# -------------------------------------------------
if not os.path.exists(CSV_PATH):
    print(f"错误: 找不到文件 {CSV_PATH}")
else:
    df = pd.read_csv(CSV_PATH)
    print("[INFO] 已加载 FairMOT 评估数据")
    print(df.describe())


[INFO] 已加载 FairMOT 评估数据
           sequence      track_id   start_frame     end_frame  track_length  \
count  51973.000000  51973.000000  51973.000000  51973.000000  51973.000000   
mean      12.157947   2245.521001    254.858734    587.184461    333.325727   
std        6.567724   2320.100442    214.812471    325.826712    265.711765   
min        0.000000      0.000000      0.000000     77.000000      2.000000   
25%        7.000000    642.000000     99.000000    313.000000    116.000000   
50%       13.000000   1437.000000    193.000000    446.000000    248.000000   
75%       19.000000   2923.000000    339.000000    836.000000    548.000000   
max       20.000000  10539.000000   1057.000000   1058.000000   1059.000000   

       num_gaps  max_gap      mean_area  max_occlusion_iou  mean_occlusion_iou  
count   51973.0  51973.0   51973.000000       51973.000000        51973.000000  
mean        0.0      0.0    8700.404938           0.824053            0.817940  
std         0.0      

In [7]:

# =================================================
# 一、整体统计分析
# =================================================
if 'df' in locals():
    def basic_statistics(df):
        stats = {
            "轨迹总数": len(df),
            "平均轨迹长度 (帧)": df["track_length"].mean(),
            "中位数轨迹长度 (帧)": df["track_length"].median(),
            "长轨迹比例 (>50帧)": (df["track_length"] > 50).mean(),
            "含断裂轨迹比例": (df["num_gaps"] > 0).mean()
        }
        return pd.Series(stats)

    basic_stats = basic_statistics(df)
    print("\n[基础统计信息]")
    print(basic_stats)



[基础统计信息]
轨迹总数            51973.000000
平均轨迹长度 (帧)        333.325727
中位数轨迹长度 (帧)       248.000000
长轨迹比例 (>50帧)        0.905393
含断裂轨迹比例             0.000000
dtype: float64


In [8]:

# =================================================
# 二、轨迹长度分布
# =================================================
if 'df' in locals():
    plt.figure(figsize=(8, 6))
    plt.hist(df["track_length"], bins=30, color='steelblue', edgecolor='black', alpha=0.8)
    plt.xlabel("轨迹长度 (帧数)")
    plt.ylabel("轨迹数量")
    plt.title("FairMOT 轨迹长度分布")
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, "track_length_distribution.png"), dpi=300)
    plt.close()


In [9]:

# =================================================
# 三、目标尺度分析
# =================================================
if 'df' in locals():
    plt.figure(figsize=(8, 6))
    plt.scatter(df["mean_area"], df["track_length"], alpha=0.6, color='mediumseagreen')
    plt.xlabel("平均目标面积 (像素)")
    plt.ylabel("轨迹长度 (帧数)")
    plt.title("目标尺度与跟踪持久性的关系")
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, "area_vs_track_length.png"), dpi=300)
    plt.close()


In [10]:

# =================================================
# 四、遮挡强度分布
# =================================================
if 'df' in locals():
    plt.figure(figsize=(8, 6))
    plt.hist(df["max_occlusion_iou"], bins=30, color='crimson', edgecolor='black', alpha=0.7)
    plt.xlabel("最大遮挡 IoU")
    plt.ylabel("轨迹数量")
    plt.title("遮挡强度分布 (Max IoU)")
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, "occlusion_distribution.png"), dpi=300)
    plt.close()


In [11]:

# =================================================
# 五、遮挡 vs 跟踪稳定性（核心）
# =================================================
if 'df' in locals():
    # 1. 遮挡 vs 长度
    plt.figure(figsize=(8, 6))
    plt.scatter(df["max_occlusion_iou"], df["track_length"], alpha=0.6, color='#8172b3')
    
    # 添加趋势线
    z = np.polyfit(df["max_occlusion_iou"], df["track_length"], 1)
    p = np.poly1d(z)
    plt.plot(df["max_occlusion_iou"], p(df["max_occlusion_iou"]), "b--", alpha=0.8, label='趋势线')
    
    plt.xlabel("最大遮挡 IoU (遮挡严重程度)")
    plt.ylabel("轨迹长度 (帧数)")
    plt.title("遮挡严重程度对跟踪长度的影响")
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, "occlusion_vs_track_length.png"), dpi=300)
    plt.close()

    # 2. 遮挡 vs 断裂
    plt.figure(figsize=(8, 6))
    plt.scatter(df["max_occlusion_iou"], df["num_gaps"], alpha=0.6, color='#ccb974')
    plt.xlabel("最大遮挡 IoU")
    plt.ylabel("轨迹断裂次数 (Gaps)")
    plt.title("遮挡严重程度对轨迹断裂的影响")
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, "occlusion_vs_gaps.png"), dpi=300)
    plt.close()


In [12]:

# =================================================
# 六、遮挡等级分析（论文非常好用）
# =================================================
if 'df' in locals():
    def occlusion_level(iou):
        if iou < 0.1:
            return "无遮挡 (None)"
        elif iou < 0.3:
            return "轻度 (Low)"
        elif iou < 0.5:
            return "中度 (Medium)"
        else:
            return "重度 (High)"

    df["occlusion_level"] = df["max_occlusion_iou"].apply(occlusion_level)

    # 指定顺序
    level_order = ["无遮挡 (None)", "轻度 (Low)", "中度 (Medium)", "重度 (High)"]
    df["occlusion_level"] = pd.Categorical(df["occlusion_level"], categories=level_order, ordered=True)

    group_stats = df.groupby("occlusion_level")[[
        "track_length", "num_gaps", "mean_area"
    ]].mean()

    print("\n[各遮挡等级统计分析]")
    print(group_stats)

    group_stats.to_csv(os.path.join(CUR_DIR, "occlusion_level_statistics.csv"))

    print("\n[INFO] 分析完成。图表已保存至:", OUT_DIR)


[各遮挡等级统计分析]
                 track_length  num_gaps    mean_area
occlusion_level                                     
无遮挡 (None)         164.837838       0.0  1293.255472
轻度 (Low)           206.180451       0.0  1734.745801
中度 (Medium)        250.704492       0.0  2305.079959
重度 (High)          337.051736       0.0  8945.123375

[INFO] 分析完成。图表已保存至: D:\Data\25-26一\图像工程\work\results_analysis\fairmot_metrics\.\figures
